In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))


import DifferentiationInterface.src.diffipy.diffipy as dp

2024-06-16 14:11:42.642236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 14:11:42.642261: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 14:11:42.642798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-16 14:11:43.040015: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc']

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance
    #dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 3, test_iterations = 10)


gradient outpt {'s0': 0.4990083906974973, 'K': -0.3626695109559818, 'r': 39.894204887147566, 'sigma': 39.85057218702792, 'dt': 7.972257026800377}
Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'s0', 'r', 'K', 'dt', 'sigma'}
numpy                10.007182    0.076368        0.000001            [0.49900837026939365, -0.36266953475916347, 39.89420489318718, 39.85057218347521, 7.972257019339678]
numpy_as_func        10.007182    0.075029        0.000001            {'s0': 0.4990083906974973, 'K': -0.3626695109559818, 'r': 39.894204887147566, 'sigma': 39.85057218702792, 'dt': 7.972257026800377}
gradient outpt [tensor(0.4988), tensor(-0.3625), tensor(39.8771), tensor(39.8354), tensor(7.9692)]
torch                10.005535    0.013321        0.000052            [0.4988265335559845, -0.3625192642211914, 39.877113342285156, 39.83542251586914, 7.969169616699219]
torch_as_func        10.005536    0.001671        0.000000            [tensor(0.4988), ten

TypeError: iif(): incompatible function arguments. The following argument types are supported:
    1. (arg0: aadc._aadc_core.ibool, arg1: aadc._aadc_core.idouble, arg2: aadc._aadc_core.idouble) -> aadc._aadc_core.idouble
    2. (arg0: aadc._aadc_core.ibool, arg1: float, arg2: aadc._aadc_core.idouble) -> aadc._aadc_core.idouble
    3. (arg0: aadc._aadc_core.ibool, arg1: aadc._aadc_core.idouble, arg2: float) -> aadc._aadc_core.idouble
    4. (arg0: bool, arg1: float, arg2: float) -> float

Invoked with: array([False,  True, False, ...,  True, False, False]), array([-49.64233591,   4.88110994, -34.99274068, ...,  13.4286832 ,
       -22.96702106, -26.03921507]), idouble(0.00e+00)

In [3]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc']

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')

    func_input_variables = [s0, K]
    diff_variables = [s0, K]

    # Record Tape: Standard Monte Carlo

    PV_standard = s0*s0*K*K

    ### Test performance
    #dp.seed(seed)
    #pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 0, test_iterations = 2)

gradient outpt {'s0': 2420000.122487545, 'K': 2200000.101327896}
Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'K', 's0'}     
numpy                121000000.000000 0.000014        0.000000            [2420000.122487545, 2200000.101327896]
numpy_as_func        121000000.000000 0.012708        0.000323            {'s0': 2420000.122487545, 'K': 2200000.101327896}
gradient outpt [tensor(2420000.), tensor(2200000.)]
torch                121000000.000000 0.000123        0.000000            [2420000.0, 2200000.0]
torch_as_func        121000000.000000 0.000028        0.000000            [tensor(2420000.), tensor(2200000.)]
gradient outpt [<tf.Tensor: shape=(), dtype=float32, numpy=2420000.0>, <tf.Tensor: shape=(), dtype=float32, numpy=2200000.0>]
tensorflow           121000000.000000 0.000565        0.000000            [2420000.0, 2200000.0]
tensorflow_as_func   121000000.000000 0.000235        0.000000            [<tf.Tensor: shape=(), dtype=floa